In [1]:
import cv2
from ultralytics import YOLO

In [2]:
yolo = YOLO('yolov10x.pt')

100%|██████████| 61.4M/61.4M [00:26<00:00, 2.43MB/s]


In [3]:
videoCap = cv2.VideoCapture(0)

In [4]:
def get_color(cls_num):
    base_colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
    color_index = cls_num % len(base_colors)
    increments = [(1, -2, 1), (-2, 1, -1), (1, -1, 2)]
    color = [base_colors[color_index][i] + increments[color_index][i] *
             (cls_num // len(base_colors)) % 256 for i in range(3)]
    return tuple(color)

In [5]:
while True:
    ret, frame = videoCap.read()
    if not ret:
        continue
    results = yolo.track(frame, stream=True)
    for result in results:
        classes_names = result.names
        for box in result.boxes:
            if box.conf[0] > 0.4:
                [x1, y1, x2, y2] = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                cls = int(box.cls[0])
                class_name = classes_names[cls]
                colour = get_color(cls)
                cv2.rectangle(frame, (x1, y1), (x2, y2), colour, 2)
                cv2.putText(frame, f'{classes_names[int(box.cls[0])]} {box.conf[0]:.2f}', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, colour, 2)

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


0: 480x640 1 person, 66.7ms
Speed: 0.9ms preprocess, 66.7ms inference, 13.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 64.4ms
Speed: 1.2ms preprocess, 64.4ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)



QObject::moveToThread: Current thread (0x1c49af40) is not the object's thread (0x1cd474e0).
Cannot move to target thread (0x1c49af40)

QObject::moveToThread: Current thread (0x1c49af40) is not the object's thread (0x1cd474e0).
Cannot move to target thread (0x1c49af40)

QObject::moveToThread: Current thread (0x1c49af40) is not the object's thread (0x1cd474e0).
Cannot move to target thread (0x1c49af40)

QObject::moveToThread: Current thread (0x1c49af40) is not the object's thread (0x1cd474e0).
Cannot move to target thread (0x1c49af40)

QObject::moveToThread: Current thread (0x1c49af40) is not the object's thread (0x1cd474e0).
Cannot move to target thread (0x1c49af40)

QObject::moveToThread: Current thread (0x1c49af40) is not the object's thread (0x1cd474e0).
Cannot move to target thread (0x1c49af40)

QObject::moveToThread: Current thread (0x1c49af40) is not the object's thread (0x1cd474e0).
Cannot move to target thread (0x1c49af40)

QObject::moveToThread: Current thread (0x1c49af40) is n

0: 480x640 1 person, 64.8ms
Speed: 0.9ms preprocess, 64.8ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 64.6ms
Speed: 1.1ms preprocess, 64.6ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 64.5ms
Speed: 1.0ms preprocess, 64.5ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 61.4ms
Speed: 1.4ms preprocess, 61.4ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 60.9ms
Speed: 1.4ms preprocess, 60.9ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 60.8ms
Speed: 0.9ms preprocess, 60.8ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 60.3ms
Speed: 0.9ms preprocess, 60.3ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 61.1ms
Speed: 0.9ms preprocess, 61.1ms inference, 0.5ms postprocess per image at shape (1, 3, 48

KeyboardInterrupt: 

In [ ]:
videoCap.release()
cv2.destroyAllWindows()